## **Jupyter Notebook for Energy Dispersive X-Ray data visualization and element quantification**
- version: EDX-release-1.1b <br>
- author: William Rigaut <br>
- date: 5.06.2024  <br>

In [ ]:
#Installing required package to run the Notebook (not needed if install.sh was executed)
#!pip install ipywidgets matplotlib tqdm nodejs npm fabio openpyxl numba
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#!mkdir data results
#!mkdir data/XRD results/XRD data/MOKE results/MOKE data/EDX results/EDX

#use this if you're using multiple versions of python on your computer (remplace 3.X by the version of python you installed)
#!python3.X -m ipykernel install --user

### **Interactive plot for EDX spectras and 2D composition maps**

In [ ]:
import os;
import re;
import pathlib;
import numpy as np;
import matplotlib.pyplot as plt;
import xml.etree.ElementTree as ET;
from openpyxl import load_workbook;
from scipy.spatial import QhullError;
from scipy.interpolate import griddata;
from ipywidgets import widgets, Layout, GridBox;
from IPython.display import display;

W_folderpath = widgets.Dropdown(
    options=[(elm, elm) for elm in os.listdir('./data/EDX/') if not elm.startswith('.')],
    value=None,
    description='Folder',
    disabled=False,
    layout=Layout(width='auto', grid_area='path')
);
W_slider_x = widgets.IntSlider(
    min=-40, max=40,
    step=5,
    description='X position',
    layout=Layout(width='auto', grid_area='X')
);
W_slider_y = widgets.IntSlider(
    min=-40, max=40,
    step=5,
    description='Y position',
    layout=Layout(width='auto', grid_area='Y')
);
W_slider_Xrange = widgets.FloatRangeSlider(
    value=[0, 20],
    min=0, max=20,
    step=.1,
    description='Energy',
    continuous_update=False,
    layout=Layout(width='auto', grid_area='Xrange')
);
W_slider_Yrange = widgets.IntRangeSlider(
    value=[0, 10000],
    min=-2000, max=50000,
    step=1000,
    description='Counts',
    continuous_update=False,
    layout=Layout(width='auto', grid_area='Yrange')
);
W_elm = widgets.Dropdown(
    options=[('Select a folder', -1)],
    value=-1,
    description='Element',
    disabled=False,
    layout=Layout(width='100%', grid_area='elm')
);
W_spectra_out = widgets.Output(
    layout=Layout(width='auto', grid_area='spectra')
);
W_resultSpectra_out = widgets.Output(
    layout=Layout(width='auto', grid_area='resultSpectra_out')
);
W_EDX_out = widgets.Output(
    layout=Layout(width='auto', grid_area='plot')
);
output = widgets.Output();

W_tabs = widgets.Tab();
tab_names = ['EDX Spectras', 'Composition Maps'];
W_tabs.children = [widgets.Text(description='filler1'),
                   widgets.Text(description='filler2')
                  ];
for i, elm in enumerate(tab_names):
    W_tabs.set_title(i, str(elm));

#interface layout and box
EDX_spectra_layout = Layout(
    width='100%',
    grid_gap='0px 0px',
    grid_template_rows='auto auto auto',
    grid_template_columns='6.66% 10% 16.66% 16.66% 16.66% 16.66% 6.66% 10%',
    grid_template_areas='''
    "path path path X X Xrange Xrange Xrange"
    ". . . Y Y Yrange Yrange Yrange"
    "spectra spectra spectra spectra spectra spectra resultSpectra_out resultSpectra_out"
    '''
);
EDX_plot_layout = Layout(
    width='100%',
    grid_gap='0px 0px',
    grid_template_rows='auto auto',
    grid_template_columns='16.66% 16.66% 16.66% 16.66% 16.66% 16.66%',
    grid_template_areas='''
    "path path elm elm . ."
    ". plot plot plot plot ."
    '''
);
EDX_spectra = GridBox(
    children=[W_folderpath, W_slider_x, W_slider_y, W_slider_Xrange, W_slider_Yrange, W_spectra_out, W_resultSpectra_out],
    layout=EDX_spectra_layout);
EDX_map = GridBox(
    children=[W_folderpath, W_elm, W_EDX_out],
    layout=EDX_plot_layout);

tab_children = [EDX_spectra, EDX_map];
W_tabs.children = tab_children;

def UpdateWidgetValue(widget):
    temp_value = widget.value;
    widget.value = None;
    widget.value = temp_value;
    return None;

def convertFloat(item):
    try:
        item = '{:.3f}'.format(float(item));
    except (ValueError, TypeError):
        pass;

    return item;

def spectra_plot(change):
    with W_spectra_out:
        W_spectra_out.clear_output();

        EDX_path = pathlib.Path(f'./data/EDX/{W_folderpath.value}');
        x_pos, y_pos = W_slider_x.value, W_slider_y.value;
        start_x, start_y = -40, -40; step = 5;
        x_idx, y_idx = int((x_pos-start_x)/step + 1), int((y_pos-start_x)/step + 1);
        filepath = pathlib.Path(f'./{EDX_path}/Spectrum_({x_idx},{y_idx}).spx');

        params = ['Atom','XLine', 'AtomPercent', 'MassPercent', 'NetIntens', 'Background', 'Sigma'];
        voltage = 0; energy_step = 0.0; zero_energy = 0.0; results_ext = []; element_list = [];
        
        tree = ET.parse(filepath);
        root = tree.getroot();
        for elm in root.iter():
            if elm.tag == 'PrimaryEnergy':
                voltage = int(float(elm.text));
            elif elm.tag == 'WorkingDistance':
                working_distance = float(elm.text);
            elif elm.tag == 'CalibLin':
                energy_step = float(elm.text);
            elif elm.tag == 'CalibAbs':
                zero_energy = float(elm.text);
            elif elm.tag == 'Channels':
                edx_spectra = np.array([((i+1)*energy_step+zero_energy, int(counts)) for i, counts in enumerate(elm.text.split(','))]);
            elif elm.tag == 'ClassInstance' and elm.get('Name') == 'Results':
                for child in elm.iter():
                    if child.tag == 'Result':
                        results_ext.append([]);
                    elif child.tag in params:
                        if child.tag == 'Atom' and int(child.text) < 10:
                            results_ext[-1].append((child.tag, f'0{child.text}'));
                        else :
                            results_ext[-1].append((child.tag, child.text));
                    elif child.tag == 'ExtResults':
                        break;
            elif elm.tag == 'ClassInstance' and elm.get('Name') == 'Elements':
                for child in elm.iter():
                    if child.get('Type') == "TRTPSEElement":
                        name_elm = child.get('Name');
                        for nb in child.iter():
                            if nb.tag == 'Element':
                                nb_elm = nb.text;
                        element_list.append([int(nb_elm), name_elm]);

        plt.figure(figsize=(15, 8));
        plt.plot([elm[0] for elm in edx_spectra], [elm[1] for elm in edx_spectra], linewidth = 0.9);
        
        plt.grid(True); plt.xlim(W_slider_Xrange.value); plt.ylim(W_slider_Yrange.value);
        plt.xlabel('Energy (keV)'); plt.ylabel('Intensity (counts)'); plt.figure(dpi=300);
        plt.show();
    with W_resultSpectra_out:
        W_resultSpectra_out.clear_output();
        print(f'Applied Voltage = {voltage} keV');
        print('Working Distance = {:.4f} mm\n'.format(working_distance));
        print('---Quantification Results---')
        for i, elm in enumerate(results_ext):
            if int(elm[0][1]) in [nb[0] for nb in element_list]:
                element_name = element_list[i][1];
            print(f'Element : {element_name}')
            for attrb in elm[1:]:
                print(f'\t{attrb[0]}={convertFloat(attrb[1])}');
    return 0;

def EDX_plot(change):
    with W_EDX_out:
        #clear the previous plot
        W_EDX_out.clear_output();
        
        #loading the data inside the .xlsx file
        datafile = pathlib.Path(f'./data/EDX/{W_folderpath.value}/Global spectrum results.xlsx');

        try:
            wb = load_workbook(filename = datafile);
        except FileNotFoundError:
            print("File not found");
            return 1;
        ws = wb.active;

        #putting all the data inside lists to obtain a treatable format
        LIST_DATA = []; X_POS = []; Y_POS = []; ELM = [];
        for i, row in enumerate(ws.values):
            LIST_DATA.append(row);
            
        #checking all the element that have been quantified by excluding the deconvoluted ones
        index_tab = [];
        for index, elm in enumerate(LIST_DATA[-3]):
            if index > 0 and elm is not None:
                if float(elm) != 0.0:
                    index_tab.append(index);
        elm_options = [(LIST_DATA[0][index], index) for index in index_tab];

        if elm_options != list(W_elm.options):
            W_elm.options = elm_options;
            return 0;
        index = W_elm.options[0][-1];
        elm_value = W_elm.value;

        #change index accordingly to prompt from user
        if index != elm_value:
            index = elm_value;
        
        #reading the data for plotting
        step_x, step_y = 5, 5;
        start_x, start_y = -40, -40;
        for row in LIST_DATA:
            if row[0] == 'Spectrum':
                title = row[index];
            elif row[0].startswith('Spectrum_'):
                x_index, y_index = row[0].split('(')[-1].split(')')[0].split(',');
                x_pos, y_pos = (int(x_index)-1)*step_x+start_x, (int(y_index)-1)*step_y+start_y;
                if np.abs(x_pos) + np.abs(y_pos) <= 60 :
                    X_POS.append(x_pos); Y_POS.append(y_pos);
                    ELM.append(float(row[index]));
            else:
                #print(row[0]);
                continue;

        #create x-y points to be used in heatmap
        xi = np.linspace(np.min(X_POS), np.max(X_POS), 500);
        yi = np.linspace(np.min(Y_POS), np.max(Y_POS), 500);
                        
        #interpolation between the points for 2D map
        try:
            zi = griddata((X_POS, Y_POS), ELM, (xi[None,:], yi[:,None]), method='cubic');
        except QhullError:
            print(f"Error: Data is not 2-dimensionnal, cannot create a map");
            return 1;
        # Create the contour plot
        plt.figure(figsize=(7.5, 6));
        plt.title(f'{title} (in at.%)');
        CS = plt.contourf(xi, yi, zi, 100, cmap=plt.cm.rainbow, vmax=max(ELM), vmin=min(ELM));
        plt.colorbar(CS);
        plt.figure(dpi=400);
        plt.show();
    return 0;

def on_tab_change(change):
    match W_tabs.selected_index:
        case 0:
            W_folderpath.observe(spectra_plot, names='value');
            W_slider_x.observe(spectra_plot, names='value');
            W_slider_y.observe(spectra_plot, names='value');
            W_slider_Xrange.observe(spectra_plot, names='value');
            W_slider_Yrange.observe(spectra_plot, names='value');
        case 1:
            W_folderpath.observe(EDX_plot, names='value');
            W_elm.observe(EDX_plot, names=['value', 'options']);

display(W_tabs, output);
W_tabs.observe(on_tab_change, names='selected_index');
W_tabs.selected_index = 1; W_tabs.selected_index = 0;

**@end-of-notebook**